In [6]:
import os
import glob
import json
import xml.etree.ElementTree as ET
from collections import defaultdict
from svgpathtools import parse_path  # pip install svgpathtools
import numpy as np
import cv2
from tqdm import tqdm

# ——— CONFIG ———
SVG_DIRS    = ["train-00", "train-01", "test-00"]
SAMPLE_STEP = 5    # SVG units per sampled point
MAX_VERTS   = 20

CLASS_MAP = {
    1: "wall",     2: "curtain wall", 3: "single door",
    4: "double door", 5: "sliding door", 9: "window",
    11: "sliding window/blind window", 12: "opening symbol",
    13: "sofa",    14: "bed",    15: "chair",
    16: "table",   17: "tv cabinet", 18: "wardrobe",
    19: "cabinet", 20: "refrigerator", 21: "airconditioner",
    22: "gas stove", 23: "sink", 24: "bath",
    25: "bath tub", 26: "washing machine", 27: "squat toilet",
    28: "urinal", 29: "toilet", 30: "stair", 31: "elevator"
}

# Pre-build COCO skeletons per directory
categories = [
    {"id": cid, "name": name, "supercategory": ""}
    for cid, name in CLASS_MAP.items()
]
coco_per_dir = {
    d: {
        "images": [],
        "annotations": [],
        "categories": categories
    }
    for d in SVG_DIRS
}
# Track IDs separately for each directory
ann_ids = {d: 1 for d in SVG_DIRS}
img_ids = {d: 1 for d in SVG_DIRS}

# Flatten the file list
all_files = []
for d in SVG_DIRS:
    for svg_path in glob.glob(os.path.join(d, "*.svg")):
        all_files.append((d, svg_path))

# Single in-place tqdm bar
with tqdm(all_files, desc="Processing SVGs") as pbar:
    for svg_dir, svg_path in pbar:
        base     = os.path.splitext(os.path.basename(svg_path))[0]
        png_path = os.path.join(svg_dir, base + ".png")
        pbar.set_description(f"{svg_dir}/{base}")

        # Parse SVG & namespace
        tree = ET.parse(svg_path)
        root = tree.getroot()
        if "}" in root.tag:
            uri       = root.tag[root.tag.find("{")+1:root.tag.find("}")]
            ns        = {"svg": uri}
            path_expr = ".//svg:path"
            viewBox   = root.get("viewBox")
        else:
            ns        = {}
            path_expr = ".//path"
            viewBox   = None

        # SVG dimensions
        if viewBox:
            _, _, svg_w, svg_h = map(float, viewBox.split())
        else:
            svg_w = float(root.get("width"))
            svg_h = float(root.get("height"))

        # Load PNG for size & scale
        img    = cv2.imread(png_path)
        h_px, w_px = img.shape[:2]
        sx, sy    = w_px / svg_w, h_px / svg_h

        # Sample points by (semantic, instance)
        groups = defaultdict(list)
        for elem in root.findall(path_expr, ns):
            sid = int(elem.get("semantic-id", -1))
            iid = int(elem.get("instance-id", -1))
            if sid not in CLASS_MAP or iid < 0:
                continue
            d = elem.get("d", "")
            if not d:
                continue
            for seg in parse_path(d):
                length = seg.length(error=1e-3)
                n_pts  = max(int(length // SAMPLE_STEP), 1)
                for i in range(n_pts + 1):
                    pt   = seg.point(i / n_pts)
                    groups[(sid, iid)].append((pt.real * sx, pt.imag * sy))

        # Image entry
        img_id    = img_ids[svg_dir]
        coco_per_dir[svg_dir]["images"].append({
            "id": img_id,
            "file_name": f"{svg_dir}/{base}.png",
            "width": w_px,
            "height": h_px
        })

        # Annotations
        ann_id = ann_ids[svg_dir]
        for (sid, iid), pts in groups.items():
            arr = np.array(pts, dtype=np.float32)
            if arr.shape[0] < 3:
                continue
            hull = cv2.convexHull(arr)
            peri = cv2.arcLength(hull, True)
            approx = hull
            for eps in np.linspace(0.001*peri, 0.1*peri, num=20):
                c = cv2.approxPolyDP(hull, eps, True)
                if len(c) <= MAX_VERTS:
                    approx = c
                    break
            coords = approx.reshape(-1, 2)
            seg = coords.flatten().tolist()
            x, y, w_box, h_box = cv2.boundingRect(coords.astype(np.int32))
            area = float(cv2.contourArea(coords))

            coco_per_dir[svg_dir]["annotations"].append({
                "id": ann_id,
                "image_id": img_id,
                "category_id": sid,
                "segmentation": [seg],
                "area": area,
                "bbox": [x, y, w_box, h_box],
                "iscrowd": 0
            })
            ann_id += 1

        img_ids[svg_dir] = img_id + 1
        ann_ids[svg_dir] = ann_id

# Write one test-00.json per directory
for d, coco in coco_per_dir.items():
    out_path = os.path.join(d, "test-00.json")
    with open(out_path, "w") as f:
        json.dump(coco, f, indent=4)
    print(f"Saved {out_path}")


test-00/0320-0032: 100%|██████████| 15663/15663 [04:17<00:00, 60.74it/s] 


Saved train-00/annotations.json
Saved train-01/annotations.json
Saved test-00/annotations.json
